#  Modeling by ML methods
1. training and risk score saving: save models for all combinations of prediction window, train window, first target focus
2. evaluation: prediction task performance, prediction timeliness

In [1]:
# module, data, feature list loading
import pandas as pd
from warnings import simplefilter
from tqdm import tqdm
import numpy as np
simplefilter(action="ignore", category=pd.errors.DtypeWarning)
pd.set_option('display.max_columns', None)

import pickle
import matplotlib.pyplot as plt
from IPython.display import clear_output

from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

from sklearn.preprocessing import MinMaxScaler as mm_scaler
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score

# feature and data loading
demo_list = ['age', 'height', 'weight', 'bmi', 'gender']# => race using postponed
vital_list = ['heart_rate', 'resp_rate', 'temperature', 'sbp', 'dbp', 'map', 'paco2', 'pao2', 'fio2']
lab_list = ['gcs', 'bilirubin', 'platelets', 'creatinine', 'lactate', 'bun', 'arterial_ph', 'wbc', 'hemoglobin', 'hematocrit','potassium', 'sodium', 'urine_output']
vaso_list = ['epinephrine', 'dopamine', 'dobutamine', 'norepinephrine', 'phenylephrine', 'vasopressin']
fl_vent_list = ['fluid', 'ventilator']

sofa_list = ['CNS_SOFA', 'CARDIO_SOFA', 'RESP_SOFA', 'COAG_SOFA', 'LIVER_SOFA', 'RENAL_SOFA', 'SOFA']
label_list = ['SEPSIS', 'SHOCK']
core_list = ['sbp', 'dbp', 'map', 'pao2', 'fio2', 'lactate', 'arterial_ph', 'gcs', 'creatinine', 'bilirubin', 'platelets']

tmp = demo_list+vital_list+lab_list+fl_vent_list+vaso_list
tmp = [i+'_value' for i in tmp if i not in ['age', 'bmi', 'gender']]
tmp += [i+'_rate' for i in vaso_list]
tmp += ['SEPSIS', 'vaso_presence']+sofa_list
base_features = tmp.copy()

statistic_features = [i+j for i in core_list for j in ['_min', '_median', '_max', '_presence']]
slope_features = [i+'_slp'+str(j) for i in core_list for j in [1,3,5]]

feature_dict = {}
vaso_fetures = {
    0 : ['SOFA'],
    1 : ['CARDIO_SOFA', 'vaso_presence'],
    2 : [i+'_rate' for i in vaso_list]+[i+'_value' for i in vaso_list]
}
for vaso_lv, fe_lv in [(a, b) for a in [0, 1, 2, 3] for b in [0, 1, 2]]:
    tmp_features = base_features.copy()
    if vaso_lv == 0:
        tmp_features = list(set(tmp_features)-set(vaso_fetures[0]+vaso_fetures[1]+vaso_fetures[2]))
    elif vaso_lv == 1:
        tmp_features = list(set(tmp_features)-set(vaso_fetures[1]+vaso_fetures[2]))
    elif vaso_lv == 2:
        tmp_features = list(set(tmp_features)-set(vaso_fetures[2]))
    
    if fe_lv == 2:
        tmp_features = tmp_features + statistic_features + slope_features
    elif fe_lv == 1:
        tmp_features = tmp_features + statistic_features

    feature_dict[f'vaso{vaso_lv}_fe{fe_lv}'] = tmp_features.copy()

curr_targets = [f'label_pw{a}_tw{b}_ftf{c}'for a in np.arange(1,13) for b in [0, 1, 2, 999] for c in [0, 1]]

icustays = pd.read_csv('processed_data/sepsis/icustays_tab_1hr_v1.csv')
icustays = icustays.loc[icustays.cohort_stays_tab_1hr_v1 == 1].reset_index(drop=True)
stayids = icustays.stay_id.tolist()
shock_ids = icustays[icustays.shock_tab_1hr_v1 == 1].stay_id.tolist()
nonshock_ids = [i for i in stayids if i not in shock_ids]

data = pd.read_csv('sepsis_data_T1FLSv3_df.csv')# with cvp # with age
data.gender = np.where(data.gender == 'F', 1, 0)
data.gender = pd.to_numeric(data.gender)
data.bmi = np.nan_to_num(data.bmi, posinf=0)

In [2]:
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.calibration import CalibratedClassifierCV

In [ ]:
import pandas as pd
from warnings import simplefilter
from tqdm import tqdm
import numpy as np
simplefilter(action="ignore", category=pd.errors.DtypeWarning)
pd.set_option('display.max_columns', None)

import pickle
import matplotlib.pyplot as plt
from IPython.display import clear_output

from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

from sklearn.preprocessing import MinMaxScaler as mm_scaler
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score

In [10]:
import sklearn, matplotlib, IPython, catboost, lightgbm, xgboost, imblearn

In [11]:
print('pandas', pd.__version__)
print('numpy', np.__version__)
print('matplotlib', matplotlib.__version__)
print('sklearn', sklearn.__version__)
print('IPython', IPython.__version__)
print('catboost', catboost.__version__)
print('lightgbm', lightgbm.__version__)
print('xgboost', xgboost.__version__)
print('imblearn', imblearn.__version__)

pandas 1.5.2
numpy 1.23.5
matplotlib 3.4.3
sklearn 1.2.0
IPython 8.10.0
catboost 1.1.1
lightgbm 3.2.1
xgboost 1.7.6
imblearn 0.10.1


## 2.1 training and risk score saving

In [2]:
def training_risk_score_saving(training_setting, data, features, risk_dict, seed):
    # training----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    pw, tw, ftf, mdl = training_setting

    target_col = [f'label_pw{pw}_tw{tw}_ftf{ftf}']
    
    trn_data = data.loc[data[f'split_{seed}']=='trn', features+target_col].dropna()

    print(f'pw: {pw}, tw: {tw}, ftf: {ftf}, model: {mdl}, seed: {seed} ==>> preparing...')
    scaler = mm_scaler()

    trn_X = trn_data.loc[:, features]
    trn_X = scaler.fit_transform(trn_X)
    trn_y = trn_data.loc[:, target_col].to_numpy().ravel()

    full_split_X = scaler.transform(data.loc[:, features].dropna())

    print(f'pw: {pw}, tw: {tw}, ftf: {ftf}, model: {mdl}, seed: {seed} ==>> training...')
    
    # initialization------------------------------
    if mdl == 'cat':
        model = CatBoostClassifier(n_estimators = 500, learning_rate = 0.2, max_depth=4, boost_from_average=False, random_state=seed, logging_level='Silent')
    elif mdl == 'xgb':
        model = XGBClassifier(n_estimators = 500, learning_rate = 0.2, max_depth=4, random_state=seed)
    elif mdl == 'lgb':
        model = LGBMClassifier(n_estimators=500, learning_rate = 0.2, max_depth=4, boost_from_average=False, random_state = seed)
    elif mdl == 'rf':
        model = RandomForestClassifier(n_estimators = 500, max_depth=4, random_state = seed)
    elif mdl == 'lr':
        model = LogisticRegression(random_state = seed)
    elif mdl == 'dt':
        model = DecisionTreeClassifier(min_samples_split=20, random_state = seed)
    elif mdl == 'nb':
        model = MultinomialNB()

    # fit------------------------------
    model.fit(trn_X, trn_y)

    key_name = f'{mdl}_pw{pw}_tw{tw}_ftf{ftf}_seed{seed}'
    risk_dict[key_name] = model.predict_proba(full_split_X)[:, 1]

    clear_output(wait=True)

In [4]:
# training, risk score saving
seed = 230726
training_settings = [(a, b, c, d) for a in np.arange(1,13) for b in [0, 1, 2, 999] for c in [0, 1] for d in ['cat', 'xgb', 'lgb', 'rf', 'lr', 'dt', 'nb']]
risk_dict={}

for trst in tqdm(training_settings):
    training_risk_score_saving(training_setting=trst, data=data.copy(), features=feature_dict[f'vaso{2}_fe{2}'], risk_dict=risk_dict, seed=seed)

100%|██████████| 672/672 [28:02:39<00:00, 150.24s/it]


In [5]:
seed = f'{seed}_wage'
with open(f'sepsis_data_risk_dict_{seed}.pkl', 'wb') as f:
    pickle.dump(risk_dict, f)
    
with open(f'sepsis_data_risk_dict_{seed}.pkl', 'rb') as f:
    risk_dict = pickle.load(f)

## 2.2 evaluation

In [12]:
seed = 230726
seed = f'{seed}_wage'
training_settings = [(a, b, c, d) for a in np.arange(1,13) for b in [0, 1, 2, 999] for c in [0, 1] for d in ['cat', 'xgb', 'lgb', 'rf', 'lr', 'dt', 'nb']]
with open(f'sepsis_data_risk_dict_{seed}.pkl', 'rb') as f:
    risk_dict = pickle.load(f)

### 2.2.1 time point based performance
1. all shock, not eval on shock
2. first shock, not eval on shock
3. all shock, eval on shock
4. first shock, eval on shock 

In [8]:
def conf_mat(true, pred):
    tp = ((pred == 1) & (true == 1)).sum()
    fp = ((pred == 1) & (true == 0)).sum()
    fn = ((pred == 0) & (true == 1)).sum()
    tn = ((pred == 0) & (true == 0)).sum()

    return tp, fp, fn, tn

def tpb_eval(training_setting, data, risk_dict, seed, eval_split='val', cutoffs=np.round(np.arange(0.01, 1, 0.01), 2)):
    # task performance evaluation---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    data = data.copy()

    pw, tw, ftf, mdl = training_setting
    
    # risk score loading
    key_name = f'{mdl}_pw{pw}_tw{tw}_ftf{ftf}_seed{seed}'
    data['risk_score'] = risk_dict[key_name]
    
    eval_result = []
    for eval_on_shock, first_onset_task in [(a, b) for a in [0, 1] for b in [0, 1]]:
        target_col = [f'label_pw{pw}_tw{999*eval_on_shock}_ftf{first_onset_task}']
        tmp_data = data.loc[data[f'split_{seed}'].isin([eval_split]), target_col+['risk_score']].dropna()
        prob = tmp_data.loc[:, 'risk_score'].values.ravel()
        true = tmp_data[target_col].values.ravel()
        
        auroc = roc_auc_score(true, prob)
        auprc = average_precision_score(true, prob)

        for cf in cutoffs:
            tmp_eval_result = [pw, tw, ftf, mdl, eval_on_shock, first_onset_task, cf]
            tmp_data['alarm'] = np.where(tmp_data['risk_score']>cf, 1, 0)
            pred = tmp_data['alarm'].values.ravel()
            
            tp, fp, fn, tn = conf_mat(true, pred)
            tpr = tp/(tp+fn)
            ppv = tp/(tp+fp) if tp+fp != 0 else 0
            f1 = 2*tp/(fp+2*tp+fn)
            tmp_eval_result += [auroc, auprc, tpr, ppv, f1]
        
            eval_result.append(tmp_eval_result)
        
    return eval_result

In [15]:
eval_results = []
seed = 230726
for trst in tqdm(training_settings):
    eval_results += tpb_eval(trst, data, risk_dict, seed)

100%|██████████| 672/672 [55:09<00:00,  4.92s/it] 


In [16]:
col_names = ['pw', 'tw', 'ftf', 'model', 'eval_during_prolonging', 'first_onset', 'cutoff', 'auroc', 'auprc', 'tpr', 'ppv', 'f1']
tpb_eval_df = pd.DataFrame(eval_results, columns=col_names)
tpb_eval_df

,pw,tw,ftf,model,eval_during_prolonging,first_onset,cutoff,auroc,auprc,tpr,ppv,f1
0,1,0,0,cat,0,0,0.01,0.951609,0.138727,0.791615,0.039173,0.074651
1,1,0,0,cat,0,0,0.02,0.951609,0.138727,0.657213,0.065463,0.119066
2,1,0,0,cat,0,0,0.03,0.951609,0.138727,0.553637,0.090689,0.155849
3,1,0,0,cat,0,0,0.04,0.951609,0.138727,0.480888,0.113570,0.183746
4,1,0,0,cat,0,0,0.05,0.951609,0.138727,0.434032,0.134095,0.204889
...,...,...,...,...,...,...,...,...,...,...,...,...
266107,12,999,1,nb,1,1,0.95,0.900351,0.521183,0.164481,0.699127,0.266309
266108,12,999,1,nb,1,1,0.96,0.900351,0.521183,0.150261,0.717465,0.248481
266109,12,999,1,nb,1,1,0.97,0.900351,0.521183,0.136751,0.744836,0.231077
266110,12,999,1,nb,1,1,0.98,0.900351,0.521183,0.115579,0.783191,0.201432


In [17]:
seed = 230726
seed = f'{seed}_wage'
tpb_eval_df.to_csv(f'processed_data/sepsis/eval_modeling/eval_tpb_{seed}.csv', index=False)

### 2.2.2 prediction timeliness performance
- EWR, EWP, EWR_stay, EWP_stay for all shock and first shock

In [19]:
def timeliness(x, criteria={'from': [-8], 'to':[0]}):
    if (x['target'].sum() == 0):
        repetition = len(criteria['from'])*len(criteria['to'])
        return np.repeat([[np.nan, np.nan, np.nan, 0, 0, x['alarm'].sum(), x['alarm'].sum()]], repetition, axis=0).ravel() # EWP 계산에만 산입
    
    else:
        if (x['alarm'].sum() == 0):
            repetition = len(criteria['from'])*len(criteria['to'])
            t_idx = np.sort(x[x['target']==1].index)

            tmp_t_idx = [t_idx[0]]
            t_onset_idx = [t_idx[0]]
            for i in t_idx[1:]:
                if (i - tmp_t_idx[-1]) > 1:
                    t_onset_idx.append(i)

                tmp_t_idx.append(i)

            return np.repeat([[0, 0, len(t_onset_idx), 0, 0, 0, 0]], repetition, axis=0).ravel() # 이 경우 (ewr, ewp)_stay = 0 
        
        else:
            t_idx = np.sort(x[x['target']==1].index)
            a_idx = np.sort(x[x['alarm']==1].index)

            oe_pairs = []
            tmp_oe = [t_idx[0]]
            for idx in t_idx:
                if (idx - tmp_oe[-1]) > 1:
                    oe_pairs.append([tmp_oe[0], tmp_oe[-1]])
                    tmp_oe = []
                tmp_oe.append(idx)
            oe_pairs.append([tmp_oe[0], tmp_oe[-1]])
            t_onset_idx = np.array(oe_pairs)[:, 0]
            t_end_idx = np.array(oe_pairs)[:, 1]

            results = []            
            for f, t in [(a, b) for a in criteria['from'] for b in criteria['to']]:
                tmp_result = [0 for _ in range(7)]
                tmp_result[2] = len(t_onset_idx)

                # all shock
                tt_num = 0
                ta = []
                for i in t_onset_idx:
                    tmp_ta = a_idx[(a_idx<(i+t))&(a_idx>=(i+f))].tolist()
                    if len(tmp_ta) > 0:
                        tt_num += 1
                        ta += tmp_ta
                ta = list(set(ta))
                tmp_result[0] = tt_num
                tmp_result[3] = len(ta)
                
                fa_idx = set(a_idx)-set(ta)
                fa_idx = set(fa_idx)-set(t_idx)
                tmp_result[5] = len(fa_idx) + len(ta)

                # first shock
                tt_num = 0
                ta = []
                tmp_ta = a_idx[(a_idx<(t_idx[0]+t))&(a_idx>=(t_idx[0]+f))].tolist()
                if len(tmp_ta) > 0:
                    tt_num += 1
                    ta += tmp_ta
                tmp_result[1] = tt_num
                tmp_result[4] = len(ta)
                tmp_result[6] = len(a_idx[a_idx<(t_idx[0]+t)])

                results += tmp_result
                
                return results

def timeliness_eval(training_setting, data, risk_dict, seed, eval_split='val', cut_offs=np.round(np.arange(0.01, 1, 0.01), 2)):
    # prediction timeliness eval
    target_col = ['SHOCK']
    data.columns = [i if i not in target_col else 'target' for i in data.columns]
        
    pw, tw, ftf, mdl = training_setting
    # risk score loading
    key_name = f'{mdl}_pw{pw}_tw{tw}_ftf{ftf}_seed{seed}'
    data['risk_score'] = risk_dict[key_name]
    tmp_data = data.loc[data[f'split_{seed}'].isin([eval_split]), ['stay_id']+['target']+['risk_score']].dropna()
    
    eval_results = []
    for cf in cut_offs:
        tmp_data['alarm'] = np.where(tmp_data['risk_score'] >= cf, 1, 0)
        ver = ['_8_0']
        
        tlns_result = pd.DataFrame(tmp_data.groupby(['stay_id']).apply(timeliness).to_list(), columns=[b+a 
                                                                                                       for a in ver 
                                                                                                       for b in ['att', 'ftt', 'at', 'ata', 'fta', 'aa', 'faa']]) 
        common_setting = [pw, tw, ftf, mdl, cf]

        for i in range(len(ver)):
            tmp_tlns_result = tlns_result.iloc[:,i*7:(i+1)*7]
            tmp = tmp_tlns_result.sum().to_numpy()
            tmp_no_na = tmp_tlns_result.dropna()
            
            tmp_eval = common_setting+['ew'+ver[i]]
            tmp_eval += [tmp[0]/tmp[2], # EWR
                         tmp[3]/tmp[5] if tmp[5] !=0 else 0, # EWP
                         tmp[1]/tmp_no_na.shape[0], # EWR_fs
                         tmp[4]/tmp[6] if tmp[6] !=0 else 0] # EWP_fs
                        
            
            tmp = tmp_no_na.eval(f'ter_stay = att{ver[i]}/at{ver[i]}').mean()
            tmp_eval += [tmp['ter_stay']]

            tmp = tmp_no_na.eval(f'tar_stay = ata{ver[i]}/aa{ver[i]}')
            tmp['tar_stay'] = [i if i >=0 else 0 for i in tmp['tar_stay']]
            tmp_eval += [tmp['tar_stay'].mean()]

            tmp = tmp_no_na.eval(f'tar_f_stay = fta{ver[i]}/faa{ver[i]}')
            tmp['tar_f_stay'] = [i if i >=0 else 0 for i in tmp['tar_f_stay']]
            tmp_eval += [tmp['tar_f_stay'].mean()]
                
            eval_results.append(tmp_eval)

    return eval_results

In [20]:
eval_results = []
for trst in tqdm(training_settings):
    eval_results += timeliness_eval(trst, data, risk_dict, seed[:-5])

100%|██████████| 672/672 [5:55:19<00:00, 31.73s/it]  


In [21]:
col_names = ['pw', 'tw', 'ftf', 'model', 'cutoff', 'eval_window','TER', 'TAR', 'TER_fs', 'TAR_fs', 'TER_stay', 'TAR_stay', 'TAR_fs_stay']
tlns_eval_df = pd.DataFrame(eval_results, columns=col_names)
tlns_eval_df.to_csv(f'processed_data/sepsis/eval_modeling/eval_tlns_{seed}.csv', index=False)
tlns_eval_df

,pw,tw,ftf,model,cutoff,eval_window,TER,TAR,TER_fs,TAR_fs,TER_stay,TAR_stay,TAR_fs_stay
0,1,0,0,cat,0.01,ew_8_0,0.914920,0.196794,0.917620,0.171053,0.917824,0.417179,0.707150
1,1,0,0,cat,0.02,ew_8_0,0.828607,0.296748,0.848970,0.256617,0.837537,0.505241,0.687150
2,1,0,0,cat,0.03,ew_8_0,0.748459,0.386470,0.766590,0.332408,0.752430,0.544093,0.657839
3,1,0,0,cat,0.04,ew_8_0,0.699137,0.458396,0.720824,0.393536,0.716421,0.575375,0.635231
4,1,0,0,cat,0.05,ew_8_0,0.662145,0.512740,0.681922,0.436494,0.679481,0.597893,0.612998
...,...,...,...,...,...,...,...,...,...,...,...,...,...
66523,12,999,1,nb,0.95,ew_8_0,0.318126,0.153646,0.215103,0.155470,0.263147,0.181751,0.166814
66524,12,999,1,nb,0.96,ew_8_0,0.279901,0.155912,0.176201,0.155351,0.229127,0.166466,0.135684
66525,12,999,1,nb,0.97,ew_8_0,0.254007,0.167462,0.155606,0.168345,0.208132,0.160706,0.123346
66526,12,999,1,nb,0.98,ew_8_0,0.214550,0.175562,0.123570,0.185950,0.173107,0.143988,0.100956


### 2.2.3 cohort based metirc

In [22]:
def cb_true_pred(x):
    positive = int(x['target'].sum() > 0)
    
    if positive == 0:
        prediction = int(x['alarm'].sum() > 0)
    else:
        first_onset = x[x['target']==1].index[0]
        prediction = int(x['alarm'].loc[:(first_onset-1)].sum() > 0)

    return positive, prediction

def conf_mat(true, pred):
    tp = ((pred == 1) & (true == 1)).sum()
    fp = ((pred == 1) & (true == 0)).sum()
    fn = ((pred == 0) & (true == 1)).sum()
    tn = ((pred == 0) & (true == 0)).sum()

    return tp, fp, fn, tn

from sklearn.metrics import auc
def cb_evaluation(training_setting, data, risk_dict, seed, eval_split='val', approx=None):
    # prediction timeliness eval
    target_col = ['SHOCK']
    data.columns = [i if i not in target_col else 'target' for i in data.columns]
        
    pw, tw, ftf, mdl = training_setting
    # risk score loading
    key_name = f'{mdl}_pw{pw}_tw{tw}_ftf{ftf}_seed{seed}'
    data['risk_score'] = risk_dict[key_name]
    tmp_data = data.loc[data[f'split_{seed}'].isin([eval_split]), ['stay_id']+['target']+['risk_score']].dropna()
    
    eval_result = []
    
    if approx == None:
        cut_offs = np.round(np.arange(0.01, 1, 0.01), 2)
    else:
        cut_offs = np.linspace(tmp_data['risk_score'].min(), tmp_data['risk_score'].max(), approx)

    for cf in tqdm(cut_offs):
        tmp_data['alarm'] = np.where(tmp_data['risk_score'] >= cf, 1, 0)
        cbm_result = pd.DataFrame(tmp_data.groupby(['stay_id']).apply(cb_true_pred).to_list(), columns=['pos', 'pred'])

        tp, fp, fn, tn = conf_mat(cbm_result['pos'], cbm_result['pred'])

        tpr = tp/(tp+fn)
        ppv = tp/(tp+fp) if tp+fp != 0 else 0
        tnr = tn/(tn+fp) if tn+fp != 0 else 0
        npv = fn/(tn+fn) if tn+fn != 0 else 0
        acc = (tp+tn)/(tp+fp+fn+tn)
        f1 = 2*tp/(fp+2*tp+fn)

        
        eval_result.append([pw, tw, ftf, mdl]+[cf, tpr, ppv, tnr, npv, acc, f1])


    tmp = np.array(eval_result)
    x, y = 1-tmp[:, 7].astype('float64'), tmp[:, 5].astype('float64')

    auroc = auc(x, y)

    eval_result = [i+[auroc] for i in eval_result]

    clear_output(wait=True)
    return eval_result

In [23]:
eval_results = []
for trst in tqdm(training_settings):
    eval_results += cb_evaluation(trst, data.copy(), risk_dict, seed[:-5])

100%|██████████| 672/672 [4:56:52<00:00, 26.51s/it]


In [24]:
col_names = ['pw', 'tw', 'ftf', 'model','cutoff', 'tpr', 'ppv', 'tnr', 'npv', 'acc', 'f1', 'auroc']
cb_eval_df = pd.DataFrame(eval_results, columns=col_names)
cb_eval_df.to_csv(f'processed_data/sepsis/eval_modeling/eval_cb_{seed}_cf.csv', index=False)
cb_eval_df

,pw,tw,ftf,model,cutoff,tpr,ppv,tnr,npv,acc,f1,auroc
0,1,0,0,cat,0.01,0.972540,0.535939,0.503374,0.031169,0.677419,0.691057,0.359397
1,1,0,0,cat,0.02,0.926773,0.574468,0.595142,0.067653,0.718166,0.709282,0.359397
2,1,0,0,cat,0.03,0.876430,0.610845,0.670715,0.098004,0.747029,0.719925,0.359397
3,1,0,0,cat,0.04,0.832952,0.646536,0.731444,0.118699,0.769100,0.728000,0.359397
4,1,0,0,cat,0.05,0.782609,0.673228,0.775978,0.141791,0.778438,0.723810,0.359397
...,...,...,...,...,...,...,...,...,...,...,...,...
66523,12,999,1,nb,0.95,0.315789,0.442308,0.765182,0.345266,0.598472,0.368491,0.536783
66524,12,999,1,nb,0.96,0.267735,0.425455,0.786775,0.354374,0.594228,0.328652,0.536783
66525,12,999,1,nb,0.97,0.233410,0.434043,0.820513,0.355249,0.602716,0.303571,0.536783
66526,12,999,1,nb,0.98,0.205950,0.476190,0.866397,0.350859,0.621392,0.287540,0.536783


### 2.2.4 further training setting exploration
1. TER > 0.9, TAR > 0.19
2. AUPRC on all shock > mean of PRC

In [25]:
import pandas as pd
seed = 230726
seed = f'{seed}_wage'
tlns_eval_df = pd.read_csv(f'processed_data/sepsis/eval_modeling/eval_tlns_{seed}.csv')
tpb_eval_df = pd.read_csv(f'processed_data/sepsis/eval_modeling/eval_tpb_{seed}.csv')

threshold = {
    'TER': 0.9,
    'TAR': 0.19, 
    'TER_fs': 0.,
    'TAR_fs': 0.
}

eval_windows = ['ew_8_0']

tlns_cand = tlns_eval_df.loc[tlns_eval_df['eval_window'].isin(eval_windows)].query(f'TER>{threshold["TER"]} & TAR>{threshold["TAR"]}')
tlns_cand_settings = tlns_cand.loc[tlns_cand.iloc[:,:4].drop_duplicates().index].iloc[:,:4]
tlns_cand_settings.shape[0]

75

In [26]:
mu_prc = tpb_eval_df.set_index(['pw', 'tw', 'ftf', 'model']).loc[tlns_cand_settings.set_index(['pw', 'tw', 'ftf', 'model']).index].query('eval_during_prolonging==0 & first_onset==0')['auprc'].mean()
mu_prc

0.23873122103605587

In [27]:
tlns_ptp_cand_settings = tpb_eval_df.set_index(['pw', 'tw', 'ftf', 'model']).loc[tlns_cand_settings.set_index(['pw', 'tw', 'ftf', 'model']).index].query('eval_during_prolonging==0 & first_onset==0').query(f'auprc>{mu_prc}').reset_index().iloc[:,:4]
tlns_ptp_cand_settings = tlns_ptp_cand_settings.drop_duplicates()
tlns_ptp_cand_settings.shape[0]

42

In [28]:
tlns_ptp_cand_settings.drop_duplicates()['model'].value_counts()

cat    33
xgb     7
lgb     2
Name: model, dtype: int64

In [29]:
tlns_ptp_cand_settings.drop_duplicates().to_csv(f'processed_data/sepsis/eval_modeling/trn_settings_{seed}_r90_p19_tpb.csv', index=False)

## 2.3 oversmapling and calibration

### 2.3.1 training and risk score saving

In [30]:
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.calibration import CalibratedClassifierCV

In [31]:
def trsv_os_cal(training_settings_df, data, features, risk_dict, seed):# TRaining Risk Score SaVing OverSampling CALibration
    # training----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    df_settings = training_settings_df.index.drop_duplicates()
    
    for pw, tw, ftf in tqdm(df_settings):
        target_col = [f'label_pw{pw}_tw{tw}_ftf{ftf}']
        
        trn_data = data.loc[data[f'split_{seed}']=='trn', features+target_col].dropna()
        cal_data = data.loc[data[f'split_{seed}']=='cal', features+target_col].dropna()

        print(f'pw: {pw}, tw: {tw}, ftf: {ftf}, seed: {seed} ==>> preparing...')
        scaler = mm_scaler()

        trn_X = trn_data.loc[:, features]
        trn_X = scaler.fit_transform(trn_X)
        trn_y = trn_data.loc[:, target_col].to_numpy().ravel()
        cal_X = cal_data.loc[:, features]
        cal_X = scaler.transform(cal_X)
        cal_y = cal_data.loc[:, target_col].to_numpy().ravel()

        oversampler = SMOTE(random_state=seed)
        trn_X_sm, trn_y_sm = oversampler.fit_resample(trn_X, trn_y)
        oversampler = ADASYN(random_state=seed)
        trn_X_ad, trn_y_ad = oversampler.fit_resample(trn_X, trn_y)

        trn_Xs = [trn_X, trn_X_sm, trn_X_ad]
        trn_ys = [trn_y, trn_y_sm, trn_y_ad]

        full_split_X = scaler.transform(data.loc[:, features].dropna())

        models = training_settings_df.loc[(pw, tw, ftf)]['model'].tolist()

        for mdl in models:
            print(f'pw: {pw}, tw: {tw}, ftf: {ftf}, model: {mdl}, seed: {seed} ==>> training...')
            
            # initialization------------------------------
            if mdl == 'cat':
                model = CatBoostClassifier(n_estimators = 500, learning_rate = 0.2, max_depth=4, boost_from_average=False, random_state=seed, logging_level='Silent')
            elif mdl == 'xgb':
                model = XGBClassifier(n_estimators = 500, learning_rate = 0.2, max_depth=4, random_state=seed)
            elif mdl == 'lgb':
                model = LGBMClassifier(n_estimators=500, learning_rate = 0.2, max_depth=4, boost_from_average=False, random_state = seed)
            elif mdl == 'rf':
                model = RandomForestClassifier(n_estimators = 500, max_depth=4, random_state = seed)
            elif mdl == 'lr':
                model = LogisticRegression(random_state = seed)
            elif mdl == 'dt':
                model = DecisionTreeClassifier(min_samples_split=20, random_state = seed)
            elif mdl == 'nb':
                model = MultinomialNB()

            # fit------------------------------
            for idx in [0, 1, 2]:
                # no cal
                model.fit(trn_Xs[idx], trn_ys[idx])
                key_name = f'{mdl}_pw{pw}_tw{tw}_ftf{ftf}_os{idx}_cal{0}_seed{seed}'
                risk_dict[key_name] = model.predict_proba(full_split_X)[:, 1]

                # sigmoid calibration
                model_sig = CalibratedClassifierCV(model, method='sigmoid', cv='prefit')
                model_sig.fit(cal_X, cal_y)
                key_name = f'{mdl}_pw{pw}_tw{tw}_ftf{ftf}_os{idx}_cal{"sig"}_seed{seed}'
                risk_dict[key_name] = model_sig.predict_proba(full_split_X)[:, 1]

                # isotonic calibration
                model_iso = CalibratedClassifierCV(model, method='isotonic', cv='prefit')
                model_iso.fit(cal_X, cal_y)
                key_name = f'{mdl}_pw{pw}_tw{tw}_ftf{ftf}_os{idx}_cal{"iso"}_seed{seed}'
                risk_dict[key_name] = model_iso.predict_proba(full_split_X)[:, 1]

        clear_output(wait=True)

In [32]:
seed = 230726
seed = f'{seed}_wage'
training_settings_df = pd.read_csv(f'processed_data/sepsis/eval_modeling/trn_settings_{seed}_r90_p19_tpb.csv')
training_settings_df = training_settings_df.set_index(['pw', 'tw', 'ftf', 'model'])

In [12]:
"""
seed = 230726
training_settings_df = pd.read_csv(f'processed_data/sepsis/eval_modeling/trn_settings_{seed}_r85_p15_ptp_v2.csv')
training_settings_df = training_settings_df.set_index(['pw', 'tw', 'ftf', 'model'])
training_settings_df_b4 = pd.read_csv(f'processed_data/sepsis/eval_modeling/trn_settings_{seed}_r85_p15_ptp.csv')
training_settings_df_b4 = training_settings_df_b4.set_index(['pw', 'tw', 'ftf', 'model'])
training_settings_df = training_settings_df.drop(list(set(training_settings_df_b4.index).intersection(set(training_settings_df.index))))
"""

In [33]:
training_settings_df = training_settings_df.reset_index().set_index(['pw', 'tw', 'ftf'])

In [35]:
risk_dict={}
seed = 230726
trsv_os_cal(training_settings_df=training_settings_df, data=data.copy(), features=feature_dict[f'vaso{2}_fe{2}'], risk_dict=risk_dict, seed=seed)

100%|██████████| 34/34 [8:26:36<00:00, 894.02s/it]


In [36]:
seed = f'{seed}_wage'
with open(f'sepsis_data_risk_dict_{seed}_r90_p19_tpb_oscal.pkl', 'wb') as f:
    pickle.dump(risk_dict, f)
    
with open(f'sepsis_data_risk_dict_{seed}_r90_p19_tpb_oscal.pkl', 'rb') as f:
    risk_dict = pickle.load(f)

In [30]:
'''
with open(f'sepsis_data_risk_dict_{seed}_oscal_r85_p15_ptp_v2.pkl', 'rb') as f:
    risk_dict = pickle.load(f)

with open(f'sepsis_data_risk_dict_{seed}_oscal_r85_p15_ptp.pkl', 'rb') as f:
    risk_dict_b4 = pickle.load(f)

key_list = [i for i in list(risk_dict_b4.keys()) if i not in list(risk_dict.keys())]
for key in tqdm(key_list):
    risk_dict[key] = risk_dict_b4[key].copy()

with open(f'sepsis_data_risk_dict_{seed}_oscal_r85_p15_ptp_v2_whole.pkl', 'wb') as f:
    pickle.dump(risk_dict, f)
'''

100%|██████████| 693/693 [00:01<00:00, 403.22it/s]


### 2.3.2 evaluation

In [37]:
seed = 230726
seed = f'{seed}_wage'
with open(f'sepsis_data_risk_dict_{seed}_r90_p19_tpb_oscal.pkl', 'rb') as f:
    risk_dict = pickle.load(f)

training_settings_df = pd.read_csv(f'processed_data/sepsis/eval_modeling/trn_settings_{seed}_r90_p19_tpb.csv')
training_settings = training_settings_df.set_index(['pw', 'tw', 'ftf', 'model']).index.tolist()
training_settings = [list(a)+[b, c] for a in training_settings for b in [0, 1, 2] for c in [0, 'sig', 'iso']]

#### 2.3.2.1 time point based

In [40]:
def conf_mat(true, pred):
    tp = ((pred == 1) & (true == 1)).sum()
    fp = ((pred == 1) & (true == 0)).sum()
    fn = ((pred == 0) & (true == 1)).sum()
    tn = ((pred == 0) & (true == 0)).sum()

    return tp, fp, fn, tn

def tpb_eval(training_setting, data, risk_dict, seed, cutoffs=np.round(np.arange(0.01, 1, 0.01), 2)):
    # task performance evaluation---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    data = data.copy()

    pw, tw, ftf, mdl, os, cal = training_setting
    
    # risk score loading
    key_name = f'{mdl}_pw{pw}_tw{tw}_ftf{ftf}_os{os}_cal{cal}_seed{seed}'
    data['risk_score'] = risk_dict[key_name]
    
    eval_result = []
    for eval_on_shock, first_onset_task in [(a, b) for a in [0, 1] for b in [0, 1]]:
        target_col = [f'label_pw{pw}_tw{999*eval_on_shock}_ftf{first_onset_task}']
        tmp_data = data.loc[data[f'split_{seed}']=='val', target_col+['risk_score']].dropna()
        prob = tmp_data.loc[:, 'risk_score'].values.ravel()
        true = tmp_data[target_col].values.ravel()
        
        auroc = roc_auc_score(true, prob)
        auprc = average_precision_score(true, prob)

        for cf in cutoffs:
            tmp_eval_result = [pw, tw, ftf, mdl, eval_on_shock, first_onset_task, cf]
            tmp_data['alarm'] = np.where(tmp_data['risk_score']>cf, 1, 0)
            pred = tmp_data['alarm'].values.ravel()
            
            tp, fp, fn, tn = conf_mat(true, pred)
            tpr = tp/(tp+fn)
            ppv = tp/(tp+fp) if tp+fp != 0 else 0
            f1 = 2*tp/(fp+2*tp+fn)
            tmp_eval_result += [auroc, auprc, tpr, ppv, f1]
        
            eval_result.append(tmp_eval_result)
        
    return eval_result

In [41]:
eval_results = []
seed = 230726
for trst in tqdm(training_settings):
    eval_results += tpb_eval(trst, data, risk_dict, seed)

100%|██████████| 378/378 [40:06<00:00,  6.37s/it]


In [42]:
col_names = ['pw', 'tw', 'ftf', 'model', 'eval_during_prolonging', 'first_onset', 'cutoff', 'auroc', 'auprc', 'tpr', 'ppv', 'f1']
tpb_eval_df = pd.DataFrame(eval_results, columns=col_names)
tpb_eval_df.to_csv(f'processed_data/sepsis/eval_modeling/eval_tpb_{seed}_r90_p19_tpb_oscal.csv', index=False)
tpb_eval_df

,pw,tw,ftf,model,eval_during_prolonging,first_onset,cutoff,auroc,auprc,tpr,ppv,f1
0,3,1,0,cat,0,0,0.01,0.940642,0.256069,0.905355,0.053352,0.100766
1,3,1,0,cat,0,0,0.02,0.940642,0.256069,0.819483,0.075916,0.138960
2,3,1,0,cat,0,0,0.03,0.940642,0.256069,0.741459,0.096969,0.171508
3,3,1,0,cat,0,0,0.04,0.940642,0.256069,0.673130,0.116902,0.199208
4,3,1,0,cat,0,0,0.05,0.940642,0.256069,0.623269,0.137938,0.225885
...,...,...,...,...,...,...,...,...,...,...,...,...
149683,12,0,0,cat,1,1,0.95,0.973416,0.860416,0.013351,0.994118,0.026349
149684,12,0,0,cat,1,1,0.96,0.973416,0.860416,0.013351,0.994118,0.026349
149685,12,0,0,cat,1,1,0.97,0.973416,0.860416,0.013351,0.994118,0.026349
149686,12,0,0,cat,1,1,0.98,0.973416,0.860416,0.013351,0.994118,0.026349


#### 2.3.2.2 prediction timeliness

In [43]:
def timeliness(x, criteria={'from': [-8], 'to':[0]}):
    if (x['target'].sum() == 0):
        repetition = len(criteria['from'])*len(criteria['to'])
        return np.repeat([[np.nan, np.nan, np.nan, 0, 0, x['alarm'].sum(), x['alarm'].sum()]], repetition, axis=0).ravel() # EWP 계산에만 산입
    
    else:
        if (x['alarm'].sum() == 0):
            repetition = len(criteria['from'])*len(criteria['to'])
            t_idx = np.sort(x[x['target']==1].index)

            tmp_t_idx = [t_idx[0]]
            t_onset_idx = [t_idx[0]]
            for i in t_idx[1:]:
                if (i - tmp_t_idx[-1]) > 1:
                    t_onset_idx.append(i)

                tmp_t_idx.append(i)

            return np.repeat([[0, 0, len(t_onset_idx), 0, 0, 0, 0]], repetition, axis=0).ravel() # 이 경우 (ewr, ewp)_stay = 0 
        
        else:
            t_idx = np.sort(x[x['target']==1].index)
            a_idx = np.sort(x[x['alarm']==1].index)

            oe_pairs = []
            tmp_oe = [t_idx[0]]
            for idx in t_idx:
                if (idx - tmp_oe[-1]) > 1:
                    oe_pairs.append([tmp_oe[0], tmp_oe[-1]])
                    tmp_oe = []
                tmp_oe.append(idx)
            oe_pairs.append([tmp_oe[0], tmp_oe[-1]])
            t_onset_idx = np.array(oe_pairs)[:, 0]
            t_end_idx = np.array(oe_pairs)[:, 1]

            results = []            
            for f, t in [(a, b) for a in criteria['from'] for b in criteria['to']]:
                tmp_result = [0 for _ in range(7)]
                tmp_result[2] = len(t_onset_idx)

                # all shock
                tt_num = 0
                ta = []
                for i in t_onset_idx:
                    tmp_ta = a_idx[(a_idx<(i+t))&(a_idx>=(i+f))].tolist()
                    if len(tmp_ta) > 0:
                        tt_num += 1
                        ta += tmp_ta
                ta = list(set(ta))
                tmp_result[0] = tt_num
                tmp_result[3] = len(ta)
                
                fa_idx = set(a_idx)-set(ta)
                fa_idx = set(fa_idx)-set(t_idx)
                tmp_result[5] = len(fa_idx) + len(ta)

                # first shock
                tt_num = 0
                ta = []
                tmp_ta = a_idx[(a_idx<(t_idx[0]+t))&(a_idx>=(t_idx[0]+f))].tolist()
                if len(tmp_ta) > 0:
                    tt_num += 1
                    ta += tmp_ta
                tmp_result[1] = tt_num
                tmp_result[4] = len(ta)
                tmp_result[6] = len(a_idx[a_idx<(t_idx[0]+t)])

                results += tmp_result
                
                return results

def timeliness_eval_oscal(training_setting, data, risk_dict, seed, cut_offs=np.round(np.arange(0.01, 1, 0.01), 2)):
    # prediction timeliness eval
    target_col = ['SHOCK']
    data.columns = [i if i not in target_col else 'target' for i in data.columns]

    pw, tw, ftf, mdl, os, cal = training_setting
    # risk score loading
    key_name = f'{mdl}_pw{pw}_tw{tw}_ftf{ftf}_os{os}_cal{cal}_seed{seed}'
    data['risk_score'] = risk_dict[key_name]
    tmp_data = data.loc[data[f'split_{seed}']=='val', ['stay_id']+['target']+['risk_score']].dropna()
    
    eval_results = []
    for cf in cut_offs:
        tmp_data['alarm'] = np.where(tmp_data['risk_score'] >= cf, 1, 0)
        ver = ['_8_0']
        
        tlns_result = pd.DataFrame(tmp_data.groupby(['stay_id']).apply(timeliness).to_list(), columns=[b+a 
                                                                                                       for a in ver
                                                                                                       for b in ['att', 'ftt', 'at', 'ata', 'fta', 'aa', 'faa']]) 
        common_setting = [pw, tw, ftf, mdl, os, cal, cf]

        for i in range(len(ver)):
            tmp_tlns_result = tlns_result.iloc[:,i*7:(i+1)*7]
            tmp = tmp_tlns_result.sum().to_numpy()
            tmp_no_na = tmp_tlns_result.dropna()
            
            tmp_eval = common_setting+['ew'+ver[i]]
            tmp_eval += [tmp[0]/tmp[2], # EWR
                         tmp[3]/tmp[5] if tmp[5] !=0 else 0, # EWP
                         tmp[1]/tmp_no_na.shape[0], # EWR_fs
                         tmp[4]/tmp[6] if tmp[6] !=0 else 0] # EWP_fs
                        
            
            tmp = tmp_no_na.eval(f'ewr_stay = att{ver[i]}/at{ver[i]}').mean()
            tmp_eval += [tmp['ewr_stay']]

            tmp = tmp_no_na.eval(f'ewp_stay = ata{ver[i]}/aa{ver[i]}')
            tmp['ewp_stay'] = [i if i >=0 else 0 for i in tmp['ewp_stay']]
            tmp_eval += [tmp['ewp_stay'].mean()]

            tmp = tmp_no_na.eval(f'ewp_f_stay = fta{ver[i]}/faa{ver[i]}')
            tmp['ewp_f_stay'] = [i if i >=0 else 0 for i in tmp['ewp_f_stay']]
            tmp_eval += [tmp['ewp_f_stay'].mean()]
                
            eval_results.append(tmp_eval)

    return eval_results

In [44]:
eval_results = []
for trst in tqdm(training_settings):
    eval_results += timeliness_eval_oscal(trst, data, risk_dict, seed)

100%|██████████| 378/378 [3:29:45<00:00, 33.29s/it]  


In [45]:
col_names = ['pw', 'tw', 'ftf', 'model', 'os', 'cal', 'cutoff', 'eval_window', 'TER', 'TAR', 'TER_fs', 'TAR_fs', 'TER_stay', 'TAR_stay', 'TAR_fs_stay']
tlns_eval_df = pd.DataFrame(eval_results, columns=col_names)
tlns_eval_df.to_csv(f'processed_data/sepsis/eval_modeling/eval_tlns_{seed}_r90_p19_tpb_oscal.csv', index=False)
tlns_eval_df

,pw,tw,ftf,model,os,cal,cutoff,eval_window,TER,TAR,TER_fs,TAR_fs,TER_stay,TAR_stay,TAR_fs_stay
0,3,1,0,cat,0,0,0.01,ew_8_0,0.966708,0.121173,0.965675,0.109104,0.965749,0.304709,0.700376
1,3,1,0,cat,0,0,0.02,ew_8_0,0.935882,0.167777,0.933638,0.152308,0.932644,0.370337,0.703643
2,3,1,0,cat,0,0,0.03,ew_8_0,0.901356,0.209043,0.906178,0.190378,0.900741,0.420333,0.705944
3,3,1,0,cat,0,0,0.04,ew_8_0,0.870530,0.247541,0.876430,0.226198,0.869443,0.448724,0.693206
4,3,1,0,cat,0,0,0.05,ew_8_0,0.831073,0.283838,0.844394,0.257631,0.828048,0.473428,0.688449
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37417,12,0,0,cat,2,iso,0.95,ew_8_0,0.017263,1.000000,0.006865,1.000000,0.013234,0.032037,0.006865
37418,12,0,0,cat,2,iso,0.96,ew_8_0,0.017263,1.000000,0.006865,1.000000,0.013234,0.032037,0.006865
37419,12,0,0,cat,2,iso,0.97,ew_8_0,0.017263,1.000000,0.006865,1.000000,0.013234,0.032037,0.006865
37420,12,0,0,cat,2,iso,0.98,ew_8_0,0.017263,1.000000,0.006865,1.000000,0.013234,0.032037,0.006865


#### 2.3.2.3 check performances

In [2]:
import pandas as pd
seed = 230726
tlns_eval_df = pd.read_csv(f'processed_data/sepsis/eval_modeling/eval_tlns_{seed}_r90_p19_tpb_oscal.csv')
#ptp_eval_df = pd.read_csv(f'processed_data/sepsis/eval_modeling/eval_ptp_{seed}_oscal_r85_p15_ptp.csv')

threshold = {
    'TER': 0.9,
    'TAR': 0.2, 
    'TER_fs': 0.,
    'TAR_fs': 0.
}

eval_windows = ['ew_8_0']

tlns_cand = tlns_eval_df.loc[tlns_eval_df['eval_window'].isin(eval_windows)].query(f'TER>{threshold["TER"]} & TAR>{threshold["TAR"]}')
tlns_cand

,pw,tw,ftf,model,os,cal,cutoff,eval_window,TER,TAR,TER_fs,TAR_fs,TER_stay,TAR_stay,TAR_fs_stay
2,3,1,0,cat,0,0,0.03,ew_8_0,0.901356,0.209043,0.906178,0.190378,0.900741,0.420333,0.705944
795,3,1,0,cat,2,iso,0.04,ew_8_0,0.901356,0.200949,0.881007,0.182215,0.888975,0.383948,0.671602
893,3,1,0,xgb,0,0,0.03,ew_8_0,0.901356,0.209295,0.897025,0.191727,0.896600,0.419564,0.699779
1490,3,1,0,xgb,2,0,0.06,ew_8_0,0.901356,0.211795,0.887872,0.192052,0.897815,0.400630,0.690616
1584,3,1,0,xgb,2,sig,0.01,ew_8_0,0.913687,0.200037,0.908467,0.181480,0.913738,0.389469,0.694317
1784,3,2,0,cat,0,0,0.03,ew_8_0,0.907522,0.207013,0.917620,0.187264,0.906843,0.418122,0.708719
1881,3,2,0,cat,0,sig,0.01,ew_8_0,0.902589,0.214403,0.913043,0.193108,0.900932,0.425033,0.704879
2675,3,2,0,xgb,0,0,0.03,ew_8_0,0.911221,0.208858,0.913043,0.190856,0.912559,0.427389,0.703960
2874,3,2,0,xgb,0,iso,0.04,ew_8_0,0.911221,0.210184,0.913043,0.192057,0.912559,0.428983,0.705086
3171,3,2,0,xgb,1,iso,0.04,ew_8_0,0.903822,0.200377,0.901602,0.182836,0.900774,0.390629,0.693172


In [144]:
import pandas as pd
seed = 230726
tlns_eval_df = pd.read_csv(f'processed_data/sepsis/eval_modeling/eval_tlns_{seed}.csv')
#ptp_eval_df = pd.read_csv(f'processed_data/sepsis/eval_modeling/eval_ptp_{seed}_oscal_r85_p15_ptp.csv')

threshold = {
    'TER': 0.8,
    'TAR': 0.2, 
    'TER_fs': 0.,
    'TAR_fs': 0.
}

eval_windows = ['ew_8_0']

tlns_cand = tlns_eval_df.loc[tlns_eval_df['eval_window'].isin(eval_windows)].query(f'TER>{threshold["TER"]} & TAR>{threshold["TAR"]} & TER_fs>{threshold["TER_fs"]} & TAR_fs>{threshold["TAR_fs"]}')
tlns_cand

,pw,tw,ftf,model,cutoff,eval_window,TER,TAR,TER_fs,TAR_fs,TER_stay,TAR_stay,TAR_fs_stay
2394,3,1,0,cat,0.05,ew_8_0,0.803946,0.249422,0.830664,0.222094,0.810232,0.466416,0.657225
2413,3,1,0,xgb,0.05,ew_8_0,0.811344,0.245622,0.832952,0.226596,0.817182,0.460781,0.666366
2432,3,1,0,lgb,0.05,ew_8_0,0.810111,0.242363,0.839817,0.222686,0.820034,0.468838,0.669165
2660,3,2,0,cat,0.05,ew_8_0,0.811344,0.229551,0.830664,0.200000,0.812676,0.456144,0.657746
2679,3,2,0,xgb,0.05,ew_8_0,0.819975,0.225695,0.842105,0.205371,0.834219,0.456010,0.667670
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11743,12,0,0,lgb,0.10,ew_8_0,0.839704,0.201992,0.862700,0.187815,0.843350,0.424273,0.666545
12104,12,1,1,cat,0.10,ew_8_0,0.805179,0.223663,0.853547,0.207458,0.824815,0.464536,0.660407
12142,12,1,1,lgb,0.10,ew_8_0,0.802713,0.203914,0.848970,0.190828,0.816272,0.444982,0.651685
12370,12,2,1,cat,0.10,ew_8_0,0.811344,0.214615,0.853547,0.198557,0.828803,0.447977,0.656197


In [150]:
tlns_cand.ftf.value_counts()/tlns_cand.shape[0]

0    0.653333
1    0.346667
Name: ftf, dtype: float64

In [25]:
tlns_cand_settings = tlns_cand.iloc[:,0:6]
tlns_cand_settings.to_csv(f'processed_data/sepsis/eval_modeling/ensemble_trn_settings_{230726}_oscal_r85_p15_ptp.csv', index=False)

## 2.4 ensemble

In [2]:
def timeliness(x, criteria={'from': [-8], 'to':[0]}):
    if (x['target'].sum() == 0):
        repetition = len(criteria['from'])*len(criteria['to'])
        return np.repeat([[np.nan, np.nan, np.nan, 0, 0, x['alarm'].sum(), x['alarm'].sum()]], repetition, axis=0).ravel() # EWP 계산에만 산입
    
    else:
        if (x['alarm'].sum() == 0):
            repetition = len(criteria['from'])*len(criteria['to'])
            t_idx = np.sort(x[x['target']==1].index)

            tmp_t_idx = [t_idx[0]]
            t_onset_idx = [t_idx[0]]
            for i in t_idx[1:]:
                if (i - tmp_t_idx[-1]) > 1:
                    t_onset_idx.append(i)

                tmp_t_idx.append(i)

            return np.repeat([[0, 0, len(t_onset_idx), 0, 0, 0, 0]], repetition, axis=0).ravel() # 이 경우 (ewr, ewp)_stay = 0 
        
        else:
            t_idx = np.sort(x[x['target']==1].index)
            a_idx = np.sort(x[x['alarm']==1].index)

            oe_pairs = []
            tmp_oe = [t_idx[0]]
            for idx in t_idx:
                if (idx - tmp_oe[-1]) > 1:
                    oe_pairs.append([tmp_oe[0], tmp_oe[-1]])
                    tmp_oe = []
                tmp_oe.append(idx)
            oe_pairs.append([tmp_oe[0], tmp_oe[-1]])
            t_onset_idx = np.array(oe_pairs)[:, 0]
            t_end_idx = np.array(oe_pairs)[:, 1]

            results = []            
            for f, t in [(a, b) for a in criteria['from'] for b in criteria['to']]:
                tmp_result = [0 for _ in range(7)]
                tmp_result[2] = len(t_onset_idx)

                # all shock
                tt_num = 0
                ta = []
                for i in t_onset_idx:
                    tmp_ta = a_idx[(a_idx<(i+t))&(a_idx>=(i+f))].tolist()
                    if len(tmp_ta) > 0:
                        tt_num += 1
                        ta += tmp_ta
                ta = list(set(ta))
                tmp_result[0] = tt_num
                tmp_result[3] = len(ta)
                
                fa_idx = set(a_idx)-set(ta)
                fa_idx = set(fa_idx)-set(t_idx)
                tmp_result[5] = len(fa_idx) + len(ta)

                # first shock
                tt_num = 0
                ta = []
                tmp_ta = a_idx[(a_idx<(t_idx[0]+t))&(a_idx>=(t_idx[0]+f))].tolist()
                if len(tmp_ta) > 0:
                    tt_num += 1
                    ta += tmp_ta
                tmp_result[1] = tt_num
                tmp_result[4] = len(ta)
                tmp_result[6] = len(a_idx[a_idx<(t_idx[0]+t)])

                results += tmp_result
                
                return results

def timeliness_eval_ens(model_settings, exp_setting, data, risk_dict, seed, voting, eval_split=['val'], cutoffs=np.round(np.arange(0.01, 1, 0.01), 2), adj_option='mm'):
    target_col = ['SHOCK']
    data.columns = [i if i not in target_col else 'target' for i in data.columns]

    print(f'{voting} ensembling....', sep=' ')
    if voting == 'hard':
        alarms = []
        for mdlst in tqdm(model_settings):
            # risk score loading
            if exp_setting.__contains__('oscal'):
                pw, tw, ftf, mdl, os, cal, cf = mdlst
                key_name = f'{mdl}_pw{pw}_tw{tw}_ftf{ftf}_os{os}_cal{cal}_seed{seed}'
            else:
                pw, tw, ftf, mdl, cf = mdlst
                key_name = f'{mdl}_pw{pw}_tw{tw}_ftf{ftf}_seed{seed}'
            
            data['risk_score'] = risk_dict[key_name]
            tmp_data = data.loc[data[f'split_{seed}'].isin(eval_split), ['risk_score']].dropna()
            tmp_data['alarm'] = np.where(tmp_data['risk_score'] >= cf, 1, 0)

            alarms.append(tmp_data['alarm'].tolist())
        
        alarms = np.array(alarms).mean(axis=0)
        print('done')

    elif voting == 'soft':
        risks = []
        for mdlst in tqdm(model_settings):
            # risk score loading
            if exp_setting.__contains__('oscal'):
                pw, tw, ftf, mdl, os, cal, cf = mdlst
                key_name = f'{mdl}_pw{pw}_tw{tw}_ftf{ftf}_os{os}_cal{cal}_seed{seed}'
            else:
                pw, tw, ftf, mdl, cf = mdlst
                key_name = f'{mdl}_pw{pw}_tw{tw}_ftf{ftf}_seed{seed}'
                
            data['risk_score'] = risk_dict[key_name]
            tmp_data = data.loc[data[f'split_{seed}'].isin(eval_split), ['risk_score']].dropna()
            risks.append(tmp_data['risk_score'].tolist())

        risks = np.array(risks)
        risk_maxs = risks.max(axis=1).reshape(-1,1)
        risk_mins = risks.min(axis=1).reshape(-1,1)
        risk_mus = risks.mean(axis=1).reshape(-1,1)
        risk_stds = risks.std(axis=1).reshape(-1,1)

        if adj_option == 'mm':
            adjusted_risks = (risks-risk_mins)/(risk_maxs-risk_mins)
        elif adj_option == 'ss':
            adjusted_risks = (risks-risk_mus)/(risk_stds)
            adjusted_risks = (adjusted_risks-adjusted_risks.min())/(adjusted_risks.max()-adjusted_risks.min())
        else:
            adjusted_risks = risks

        adjusted_risks = adjusted_risks.mean(axis=0)

        print('done')

    else:
        return 'error! error!'

    eval_results = []
    tmp_data = data.loc[data[f'split_{seed}'].isin(eval_split), ['stay_id', 'target', 'risk_score']].dropna()

    print('evaluating....', sep=' ')
    for cf in tqdm(cutoffs):
        if voting == 'hard':
            tmp_data['alarm'] = np.where(alarms > cf, 1, 0)
        elif voting == 'soft':
            tmp_data['risk_score'] = adjusted_risks
            tmp_data['alarm'] = np.where(tmp_data['risk_score'] > cf, 1, 0)

        ver = ['_8_0']

        tlns_result = pd.DataFrame(tmp_data.groupby(['stay_id']).apply(timeliness).to_list(), columns=[b+a 
                                                                                                        for a in ver 
                                                                                                        for b in ['att', 'ftt', 'at', 'ata', 'fta', 'aa', 'faa']]) 
        

        for i in range(len(ver)):
            tmp_tlns_result = tlns_result.iloc[:,i*7:(i+1)*7]
            tmp = tmp_tlns_result.sum().to_numpy()
            tmp_no_na = tmp_tlns_result.dropna()
            
            tmp_eval = [cf, 'ew'+ver[i]]
            tmp_eval += [tmp[0]/tmp[2], # EWR
                            tmp[3]/tmp[5] if tmp[5] !=0 else 0, # EWP
                            tmp[1]/tmp_no_na.shape[0], # EWR_fs
                            tmp[4]/tmp[6] if tmp[6] !=0 else 0] # EWP_fs
                        
            
            tmp = tmp_no_na.eval(f'ewr_stay = att{ver[i]}/at{ver[i]}').mean()
            tmp_eval += [tmp['ewr_stay']]

            tmp = tmp_no_na.eval(f'ewp_stay = ata{ver[i]}/aa{ver[i]}')
            tmp['ewp_stay'] = [i if i >=0 else 0 for i in tmp['ewp_stay']]
            tmp_eval += [tmp['ewp_stay'].mean()]

            tmp = tmp_no_na.eval(f'ewp_f_stay = fta{ver[i]}/faa{ver[i]}')
            tmp['ewp_f_stay'] = [i if i >=0 else 0 for i in tmp['ewp_f_stay']]
            tmp_eval += [tmp['ewp_f_stay'].mean()]
                
            eval_results.append(tmp_eval)

    print('done')
    return eval_results

In [3]:
seed = 230726
seed = f'{seed}_wage'
exp_setting= '_r90_p19_tpb_oscal'

with open(f'sepsis_data_risk_dict_{seed}{exp_setting}.pkl', 'rb') as f:
    risk_dict = pickle.load(f)
    
seed = 230726
tlns_eval_df = pd.read_csv(f'processed_data/sepsis/eval_modeling/eval_tlns_{seed}{exp_setting}.csv')

In [4]:
threshold = {
    'TER': 0.9,
    'TAR': 0.2
}

eval_windows = ['ew_8_0']

tlns_cand = tlns_eval_df.loc[tlns_eval_df['eval_window'].isin(eval_windows)].query(f'TER>{threshold["TER"]} & TAR>{threshold["TAR"]}')
tlns_cand.loc[tlns_cand['TER'].idxmax()]
ter_max = tlns_cand.loc[tlns_cand['TER'].idxmax()]['TER']

In [14]:
threshold = {
    'TER': 0.9,
    'TAR': 0.2
}
eval_windows = ['ew_8_0']
tlns_cand = tlns_eval_df.loc[tlns_eval_df['eval_window'].isin(eval_windows)].query(f'TER>{threshold["TER"]} & TAR>{threshold["TAR"]}')
if exp_setting.__contains__('oscal'):
    model_settings = tlns_cand.loc[:, ['pw', 'tw', 'ftf', 'model', 'os', 'cal', 'cutoff']].drop_duplicates().to_numpy()
else:
    model_settings = tlns_cand.loc[:, ['pw', 'tw', 'ftf', 'model', 'cutoff']].drop_duplicates().to_numpy()

In [6]:
eval_result = timeliness_eval_ens(model_settings, exp_setting, data.copy(), risk_dict, seed, eval_split=['val'], voting='hard', adj_option='mm')

col_names = ['cutoff', 'eval_window', 'TER', 'TAR', 'TER_fs', 'TAR_fs', 'TER_stay', 'TAR_stay', 'TAR_fs_stay']
ens_eval_df = pd.DataFrame(eval_result, columns=col_names)
ens_eval_df.loc[ens_eval_df['eval_window'].isin(eval_windows)].query(f'TER>{ter_max} & TAR>{max(threshold["TAR"], 0.2)}')

hard ensembling....


100%|██████████| 149/149 [00:40<00:00,  3.68it/s]


done
evaluating....


100%|██████████| 99/99 [00:33<00:00,  2.99it/s]

done


,cutoff,eval_window,TER,TAR,TER_fs,TAR_fs,TER_stay,TAR_stay,TAR_fs_stay
42,0.43,ew_8_0,0.917386,0.201018,0.919908,0.186127,0.917653,0.407659,0.705132
43,0.44,ew_8_0,0.914920,0.201969,0.919908,0.187139,0.916754,0.409602,0.705411


In [15]:
eval_result = timeliness_eval_ens(model_settings, exp_setting, data.copy(), risk_dict, seed, eval_split=['tst'], voting='hard', adj_option='ss')

col_names = ['cutoff', 'eval_window', 'TER', 'TAR', 'TER_fs', 'TAR_fs', 'TER_stay', 'TAR_stay', 'TAR_fs_stay']
ens_eval_df = pd.DataFrame(eval_result, columns=col_names)
ens_eval_df.loc[ens_eval_df['eval_window'].isin(eval_windows)].query(f'TER>{tlns_cand["TER"].max()} & TAR>{max(threshold["TAR"], 0.2)}')

hard ensembling....


100%|██████████| 40/40 [00:11<00:00,  3.34it/s]


done
evaluating....


100%|██████████| 99/99 [00:33<00:00,  2.92it/s]

done


,cutoff,eval_window,TER,TAR,TER_fs,TAR_fs,TER_stay,TAR_stay,TAR_fs_stay
27,0.28,ew_8_0,0.940299,0.201756,0.931350,0.178388,0.934650,0.430463,0.771734
28,0.29,ew_8_0,0.940299,0.201756,0.931350,0.178388,0.934650,0.430463,0.771734
29,0.30,ew_8_0,0.937100,0.206035,0.929062,0.182823,0.932607,0.436550,0.771783
30,0.31,ew_8_0,0.937100,0.206035,0.929062,0.182823,0.932607,0.436550,0.771783
31,0.32,ew_8_0,0.937100,0.206035,0.929062,0.182823,0.932607,0.436550,0.771783
32,0.33,ew_8_0,0.932836,0.210161,0.926773,0.187619,0.928412,0.441481,0.773742
33,0.34,ew_8_0,0.932836,0.210161,0.926773,0.187619,0.928412,0.441481,0.773742
34,0.35,ew_8_0,0.928571,0.214029,0.924485,0.191779,0.923454,0.445446,0.773287
35,0.36,ew_8_0,0.928571,0.214029,0.924485,0.191779,0.923454,0.445446,0.773287
36,0.37,ew_8_0,0.928571,0.214029,0.924485,0.191779,0.923454,0.445446,0.773287


In [16]:
ens_eval_df.to_csv(f'processed_data/sepsis/eval_modeling/eval_tlns_{seed}{exp_setting}.csv', index=False)

In [50]:
ens_eval_df.to_csv('tmp.csv', index=False)

### alarm before vasopressor

In [18]:
def abv_eval(x, criteria={'from': [-8], 'to':[0]}):
    if (x['target'].sum() == 0):
        return [np.nan for _ in range(5)] # EWP 계산에만 산입
    
    else:
        if (x['alarm'].sum() == 0):
            return [0 for _ in range(4)]+[len(t_onset_idx)] # 이 경우 (ewr, ewp)_stay = 0 
        
        else:
            t_idx = np.sort(x[x['target']==1].index)
            a_idx = np.sort(x[x['alarm']==1].index)
            v_idx = np.sort(x[x['vaso_presence']==1].index) 
            f_idx = np.sort(x[x['fluid_value']>0].index) 

            oe_pairs = []
            tmp_oe = [t_idx[0]]
            for idx in t_idx:
                if (idx - tmp_oe[-1]) > 1:
                    oe_pairs.append([tmp_oe[0], tmp_oe[-1]])
                    tmp_oe = []
                tmp_oe.append(idx)
            oe_pairs.append([tmp_oe[0], tmp_oe[-1]])
            t_onset_idx = np.array(oe_pairs)[:, 0]

            results = []
            for f, t in [(a, b) for a in criteria['from'] for b in criteria['to']]:
                tmp_result = [0 for _ in range(5)]
                tmp_result[4] = len(t_onset_idx)

                # all shock
                for i in t_onset_idx:
                    tmp_ta = a_idx[(a_idx<(i+t))&(a_idx>=(i+f))].tolist()
                    tmp_v = v_idx[(v_idx<(i+t))&(v_idx>=(i+f))].tolist()
                    tmp_f = f_idx[(f_idx<(i+t))&(f_idx>=(i+f))].tolist()
                    if len(tmp_ta) > 0:
                        if (len(tmp_v) > 0) & (len(tmp_f) > 0):
                            tmp_vf = [i for i in v_idx if i in f_idx]
                            if len(tmp_vf) == 0:
                                tmp_result[0] += 1
                                tmp_result[1] += 1
                            else:
                                tmp_ta.sort()
                                tmp_vf.sort()
                                if tmp_ta[0] <= tmp_vf[0]:
                                    tmp_result[0] += 1
                                    if tmp_ta[0] < tmp_vf[0]:
                                        tmp_result[1] += 1
                        else:
                            tmp_result[0] += 1
                            tmp_result[1] += 1
                            if (len(tmp_v) > 0):
                                tmp_ta.sort()
                                tmp_v.sort()
                                if tmp_ta[0] <= tmp_v[0]:
                                    tmp_result[2] += 1
                                    if tmp_ta[0] < tmp_v[0]:
                                        tmp_result[3] += 1
                            else:
                                tmp_result[2] += 1
                                tmp_result[3] += 1

                results += tmp_result
                
                return results

def abv_eval_ens(model_settings, exp_setting, data, risk_dict, seed, voting, cutoff, eval_split=['val'], adj_option=None):
    target_col = ['SHOCK']
    data.columns = [i if i not in target_col else 'target' for i in data.columns]

    print(f'{voting} ensembling....', sep=' ')
    if voting == 'hard':
        alarms = []
        for mdlst in tqdm(model_settings):
            # risk score loading
            if exp_setting.__contains__('oscal'):
                pw, tw, ftf, mdl, os, cal, cf = mdlst
                key_name = f'{mdl}_pw{pw}_tw{tw}_ftf{ftf}_os{os}_cal{cal}_seed{seed}'
            else:
                pw, tw, ftf, mdl, cf = mdlst
                key_name = f'{mdl}_pw{pw}_tw{tw}_ftf{ftf}_seed{seed}'
            
            data['risk_score'] = risk_dict[key_name]
            tmp_data = data.loc[data[f'split_{seed}'].isin(eval_split), ['risk_score']].dropna()
            tmp_data['alarm'] = np.where(tmp_data['risk_score'] >= cf, 1, 0)

            alarms.append(tmp_data['alarm'].tolist())
        
        alarms = np.array(alarms).mean(axis=0)
        print('done')

    elif voting == 'soft':
        risks = []
        for mdlst in tqdm(model_settings):
            # risk score loading
            if exp_setting.__contains__('oscal'):
                pw, tw, ftf, mdl, os, cal, cf = mdlst
                key_name = f'{mdl}_pw{pw}_tw{tw}_ftf{ftf}_os{os}_cal{cal}_seed{seed}'
            else:
                pw, tw, ftf, mdl, cf = mdlst
                key_name = f'{mdl}_pw{pw}_tw{tw}_ftf{ftf}_seed{seed}'
                
            data['risk_score'] = risk_dict[key_name]
            tmp_data = data.loc[data[f'split_{seed}'].isin(eval_split), ['risk_score']].dropna()
            risks.append(tmp_data['risk_score'].tolist())

        risks = np.array(risks)
        risk_maxs = risks.max(axis=1).reshape(-1,1)
        risk_mins = risks.min(axis=1).reshape(-1,1)
        risk_mus = risks.mean(axis=1).reshape(-1,1)
        risk_stds = risks.std(axis=1).reshape(-1,1)

        if adj_option == 'mm':
            adjusted_risks = (risks-risk_mins)/(risk_maxs-risk_mins)
        elif adj_option == 'ss':
            adjusted_risks = (risks-risk_mus)/(risk_stds)
            adjusted_risks = (adjusted_risks-adjusted_risks.min())/(adjusted_risks.max()-adjusted_risks.min())
        else:
            adjusted_risks = risks

        adjusted_risks = adjusted_risks.mean(axis=0)

        print('done')

    else:
        return 'error! error!'

    
    tmp_data = data.loc[data[f'split_{seed}'].isin(eval_split), ['stay_id', 'target', 'risk_score', 'vaso_presence', 'fluid_value']].dropna()

    print('evaluating....', sep=' ')

    cf = cutoff
    if voting == 'hard':
        tmp_data['alarm'] = np.where(alarms > cf, 1, 0)
    elif voting == 'soft':
        tmp_data['risk_score'] = adjusted_risks
        tmp_data['alarm'] = np.where(tmp_data['risk_score'] > cf, 1, 0)
    abv_result = pd.DataFrame(tmp_data.groupby(['stay_id']).apply(abv_eval).to_list(), columns=['tt_vf_ee', 'tt_vf_e', 'tt_v_ee', 'tt_v_e', 'at']) 
    
    eval_results = [abv_result.sum()[i]/abv_result.sum()[-1] for i in range(4)]

    print('done')
    return eval_results

In [17]:
threshold = {
    'TER': 0.9,
    'TAR': 0.2
}

eval_windows = ['ew_8_0']

tlns_cand = tlns_eval_df.loc[tlns_eval_df['eval_window'].isin(eval_windows)].query(f'TER>{threshold["TER"]} & TAR>{threshold["TAR"]}')
if exp_setting.__contains__('oscal'):
    model_settings = tlns_cand.loc[:, ['pw', 'tw', 'ftf', 'model', 'os', 'cal', 'cutoff']].drop_duplicates().to_numpy()
else:
    model_settings = tlns_cand.loc[:, ['pw', 'tw', 'ftf', 'model', 'cutoff']].drop_duplicates().to_numpy()
tlns_cand

,pw,tw,ftf,model,os,cal,cutoff,eval_window,TER,TAR,TER_fs,TAR_fs,TER_stay,TAR_stay,TAR_fs_stay
2,3,1,0,cat,0,0,0.03,ew_8_0,0.901356,0.209043,0.906178,0.190378,0.900741,0.420333,0.705944
795,3,1,0,cat,2,iso,0.04,ew_8_0,0.901356,0.200949,0.881007,0.182215,0.888975,0.383948,0.671602
893,3,1,0,xgb,0,0,0.03,ew_8_0,0.901356,0.209295,0.897025,0.191727,0.896600,0.419564,0.699779
1490,3,1,0,xgb,2,0,0.06,ew_8_0,0.901356,0.211795,0.887872,0.192052,0.897815,0.400630,0.690616
1584,3,1,0,xgb,2,sig,0.01,ew_8_0,0.913687,0.200037,0.908467,0.181480,0.913738,0.389469,0.694317
1784,3,2,0,cat,0,0,0.03,ew_8_0,0.907522,0.207013,0.917620,0.187264,0.906843,0.418122,0.708719
1881,3,2,0,cat,0,sig,0.01,ew_8_0,0.902589,0.214403,0.913043,0.193108,0.900932,0.425033,0.704879
2675,3,2,0,xgb,0,0,0.03,ew_8_0,0.911221,0.208858,0.913043,0.190856,0.912559,0.427389,0.703960
2874,3,2,0,xgb,0,iso,0.04,ew_8_0,0.911221,0.210184,0.913043,0.192057,0.912559,0.428983,0.705086
3171,3,2,0,xgb,1,iso,0.04,ew_8_0,0.903822,0.200377,0.901602,0.182836,0.900774,0.390629,0.693172


In [19]:
eval_result = abv_eval_ens(model_settings, exp_setting, data.copy(), risk_dict, seed, eval_split=['tst'], voting='hard', cutoff=0.28)
eval_result

hard ensembling....


100%|██████████| 40/40 [00:14<00:00,  2.79it/s]


done
evaluating....
done


[0.5063965884861408,
 0.48507462686567165,
 0.43390191897654584,
 0.43390191897654584]

## etc

In [21]:
def high_corr_check(x, criteria={'from': -4, 'to': 0, 'irvsdr':24}, corr_type='pearson'):
    from numpy import dot
    from numpy.linalg import norm

    def cosine(a, b):
        return dot(a, b)/(norm(a)*norm(b))

    t_idx = np.sort(x[x['target']==1].index)
    risk = x['risk_score']

    tmp_t_idx = [t_idx[0]] 
    dr_t_idx = []
    ir_t_idx = []
    for i in t_idx[1:]:
        if (i - tmp_t_idx[-1]) > 1:
            if (i - tmp_t_idx[-1]) > criteria['irvsdr']:
                dr_t_idx.append(i)
            else:
                ir_t_idx.append(i)
                
        tmp_t_idx.append(i)

    results = []
    f = criteria['from']
    t = criteria['to']

    # first shock
    first_risk = risk.loc[np.max((x.index[0], t_idx[0]+f)):(t_idx[0]+t-1)].tolist()
    
    if len(first_risk) < 2:
        return np.repeat(np.nan, 8)
    else:

        # delayed shock
        delayed_risks = []
        for i in dr_t_idx:
            tmp_d_risk = risk.loc[np.max((i-len(first_risk), i+f)):(i+t-1)].tolist()
            delayed_risks.append(tmp_d_risk)

        # immediate shock
        immediate_risks = []
        for i in ir_t_idx:
            tmp_i_risk = risk.loc[np.max((i-len(first_risk), i+f)):(i+t-1)].tolist()
            immediate_risks.append(tmp_i_risk)

        if (len(delayed_risks) == 0)&(len(immediate_risks) == 0):
            return np.repeat(np.nan, 8)
        
        elif (len(delayed_risks) == 0):
        # correlation calculation
            risks_bfs = [first_risk]
            risks_bfs += immediate_risks
            risks_bfs = np.vstack(risks_bfs)

            if corr_type=='pearson':
                corrs = np.corrcoef(np.vstack(risks_bfs))[0][1:]
                
            elif corr_type=='euclidean':
                max_d = len(risks_bfs[0])
                corrs = np.array([(max_d-norm(risks_bfs[0]-i))/max_d for i in risks_bfs[1:]])
                
            if corr_type=='cosine':
                corrs = np.array([cosine(risks_bfs[0], i) for i in risks_bfs[1:]])

            
            results += [len(corrs[corrs > 0.1]) / len(corrs),
                        len(corrs[corrs > 0.3]) / len(corrs),
                        len(corrs[corrs > 0.5]) / len(corrs),
                        len(corrs[corrs > 0.7]) / len(corrs)]
            
            return [np.nan for _ in range(4)]+results
        
        elif (len(immediate_risks) == 0):
        # correlation calculation
            risks_bfs = [first_risk]
            risks_bfs += delayed_risks
            risks_bfs = np.vstack(risks_bfs)

            if corr_type=='pearson':
                corrs = np.corrcoef(np.vstack(risks_bfs))[0][1:]
                
            elif corr_type=='euclidean':
                max_d = len(risks_bfs[0])
                corrs = np.array([(max_d-norm(risks_bfs[0]-i))/max_d for i in risks_bfs[1:]])
                
            if corr_type=='cosine':
                corrs = np.array([cosine(risks_bfs[0], i) for i in risks_bfs[1:]])
            
            results += [len(corrs[corrs > 0.1]) / len(corrs),
                        len(corrs[corrs > 0.3]) / len(corrs),
                        len(corrs[corrs > 0.5]) / len(corrs),
                        len(corrs[corrs > 0.7]) / len(corrs)]

            return results+[np.nan for _ in range(4)]
        
        else:
            # correlation calculation
            for r in [delayed_risks, immediate_risks]:
                risks_bfs = [first_risk]
                risks_bfs += r
                risks_bfs = np.vstack(risks_bfs)

                if corr_type=='pearson':
                    corrs = np.corrcoef(np.vstack(risks_bfs))[0][1:]
                    
                elif corr_type=='euclidean':
                    max_d = len(risks_bfs[0])
                    corrs = np.array([(max_d-norm(risks_bfs[0]-i))/max_d for i in risks_bfs[1:]])
                    
                if corr_type=='cosine':
                    corrs = np.array([cosine(risks_bfs[0], i) for i in risks_bfs[1:]])
                
                
                results += [len(corrs[corrs > 0.1]) / len(corrs),
                            len(corrs[corrs > 0.3]) / len(corrs),
                            len(corrs[corrs > 0.5]) / len(corrs),
                            len(corrs[corrs > 0.7]) / len(corrs)]

            return results

def corr_btw_fs_dr(risk_dict, seed, setting, data, corr_type='pearson', irvsdr=24, k_cv=5):
    simplefilter(action="ignore", category=RuntimeWarning)
    target_col = ['SHOCK']
    data.columns = [i if i not in target_col else 'target' for i in data.columns]
    
    pw, trdc, mdl, sm, cal, fold_idx = setting
    # risk score loading
    risk_key = f'{mdl}_pw{pw}_trdc{trdc}_f{fold_idx}_sm{sm}_cal{cal}_seed{seed}'
    data['risk_score'] = risk_dict[risk_key]
    tmp_data = data.loc[data[f'{k_cv}cv_fold_{seed}']==fold_idx, ['stay_id']+['target']+['risk_score']].dropna()
    shock_ids = tmp_data.loc[tmp_data.target == 1].stay_id.unique()
    tmp_data = tmp_data.loc[tmp_data.stay_id.isin(shock_ids)]

    # high corr check btw fs and dr
    corr_result = tmp_data.groupby(['stay_id']).apply(partial(high_corr_check, corr_type=corr_type, criteria={'from': -4, 'to': 0, 'irvsdr':irvsdr})).to_list()
    corr_result_w_setting = [pw, trdc, mdl, sm, cal, fold_idx]+pd.DataFrame(corr_result).iloc[:,-4:].dropna().mean().tolist()+pd.DataFrame(corr_result).iloc[:,-8:-4].dropna().mean().tolist()
    
    return corr_result_w_setting

prediction_window = np.arange(1, 7)
train_crop = np.hstack([np.arange(0, 7), 12])
k_cv = 5
model_name = ['xgb', 'cat']
training_settings = [(a,b,c,d,e,f) 
                     for a in prediction_window 
                     for b in train_crop 
                     for c in model_name 
                     for d in range(2) 
                     for e in range(2)
                     for f in range(k_cv)]


In [ ]:
eval_data = []
for i in tqdm(training_settings):
    eval_data.append(corr_btw_fs_dr(setting=i, seed=seed, risk_dict=risk_dict.copy(), data=df.copy(), k_cv=k_cv))
col_names = ['pwin', 'trdc', 'mdl', 'sm', 'cal', 'fold']+[b+a for a in ['_40_dr', '_40_ir'] for b in ['weak', 'moder', 'strong', 'vstrng']]
new_eval_data = pd.DataFrame(eval_data, columns=col_names)
f_name = f'processed_data/sepsis/eval_modeling/exp_corr_btw_fs_r_{seed}_pc.csv'
new_eval_data.to_csv(f_name, index=False)

new_eval_data = pd.read_csv(f_name)
new_eval_data[col_names[-8:]].astype(float).mean()

100%|██████████| 1920/1920 [22:11<00:00,  1.44it/s]


weak_40_dr      0.436999
moder_40_dr     0.365012
strong_40_dr    0.289720
vstrng_40_dr    0.211579
weak_40_ir      0.474992
moder_40_ir     0.398776
strong_40_ir    0.315526
vstrng_40_ir    0.227469
dtype: float64

In [ ]:
eval_data = []
for i in tqdm(training_settings):
    eval_data.append(corr_btw_fs_dr(setting=i, seed=seed, risk_dict=risk_dict.copy(), data=df.copy(), corr_type='euclidean', k_cv=k_cv))

col_names = ['pwin', 'trdc', 'mdl', 'sm', 'cal', 'fold']+[b+a for a in ['_40_dr', '_40_ir'] for b in ['weak', 'moder', 'strong', 'vstrng']]
new_eval_data = pd.DataFrame(eval_data, columns=col_names)
f_name = f'processed_data/sepsis/eval_modeling/exp_corr_btw_fs_r_{seed}.csv'
new_eval_data.to_csv(f_name, index=False)

new_eval_data = pd.read_csv(f_name)
new_eval_data[col_names[-8:]].astype(float).mean()

100%|██████████| 1920/1920 [21:06<00:00,  1.52it/s]


weak_40_dr      1.000000
moder_40_dr     0.999364
strong_40_dr    0.979050
vstrng_40_dr    0.823113
weak_40_ir      1.000000
moder_40_ir     0.999039
strong_40_ir    0.978545
vstrng_40_ir    0.814504
dtype: float64

In [ ]:
eval_data = []
for i in tqdm(training_settings):
    eval_data.append(corr_btw_fs_dr(setting=i, seed=seed, risk_dict=risk_dict.copy(), data=df.copy(), corr_type='cosine', k_cv=k_cv))

col_names = ['pwin', 'trdc', 'mdl', 'sm', 'cal', 'fold']+[b+a for a in ['_40_dr', '_40_ir'] for b in ['weak', 'moder', 'strong', 'vstrng']]
new_eval_data = pd.DataFrame(eval_data, columns=col_names)
f_name = f'processed_data/sepsis/eval_modeling/exp_corr_btw_fs_r_{seed}_cos.csv'
new_eval_data.to_csv(f_name, index=False)

new_eval_data = pd.read_csv(f_name)
new_eval_data[col_names[-8:]].astype(float).mean()

100%|██████████| 1920/1920 [21:18<00:00,  1.50it/s]


weak_40_dr      0.991867
moder_40_dr     0.962326
strong_40_dr    0.896866
vstrng_40_dr    0.733325
weak_40_ir      0.996614
moder_40_ir     0.977553
strong_40_ir    0.924279
vstrng_40_ir    0.762686
dtype: float64

In [22]:
eval_data = []
irvsdr = 12
for i in tqdm(training_settings):
    eval_data.append(corr_btw_fs_dr(setting=i, seed=seed, risk_dict=risk_dict.copy(), data=df.copy(), irvsdr=irvsdr, k_cv=k_cv))
col_names = ['pwin', 'trdc', 'mdl', 'sm', 'cal', 'fold']+[b+a for a in ['_40_dr', '_40_ir'] for b in ['weak', 'moder', 'strong', 'vstrng']]
new_eval_data = pd.DataFrame(eval_data, columns=col_names)
f_name = f'processed_data/sepsis/eval_modeling/exp_corr_btw_fs_r_{seed}_pc_irvsdr{irvsdr}.csv'
new_eval_data.to_csv(f_name, index=False)

new_eval_data = pd.read_csv(f_name)
new_eval_data[col_names[-8:]].astype(float).mean()

100%|██████████| 1920/1920 [22:07<00:00,  1.45it/s]


weak_40_dr      0.438690
moder_40_dr     0.367770
strong_40_dr    0.293076
vstrng_40_dr    0.214929
weak_40_ir      0.462652
moder_40_ir     0.386378
strong_40_ir    0.304832
vstrng_40_ir    0.218951
dtype: float64

In [23]:
eval_data = []
for i in tqdm(training_settings):
    eval_data.append(corr_btw_fs_dr(setting=i, seed=seed, risk_dict=risk_dict.copy(), data=df.copy(), corr_type='euclidean', irvsdr=irvsdr, k_cv=k_cv))

col_names = ['pwin', 'trdc', 'mdl', 'sm', 'cal', 'fold']+[b+a for a in ['_40_dr', '_40_ir'] for b in ['weak', 'moder', 'strong', 'vstrng']]
new_eval_data = pd.DataFrame(eval_data, columns=col_names)
f_name = f'processed_data/sepsis/eval_modeling/exp_corr_btw_fs_r_{seed}_euc_irvsdr{irvsdr}.csv'
new_eval_data.to_csv(f_name, index=False)

new_eval_data = pd.read_csv(f_name)
new_eval_data[col_names[-8:]].astype(float).mean()

100%|██████████| 1920/1920 [21:05<00:00,  1.52it/s]


weak_40_dr      1.000000
moder_40_dr     0.999280
strong_40_dr    0.977403
vstrng_40_dr    0.817511
weak_40_ir      1.000000
moder_40_ir     0.999174
strong_40_ir    0.979872
vstrng_40_ir    0.822139
dtype: float64

In [24]:
eval_data = []
for i in tqdm(training_settings):
    eval_data.append(corr_btw_fs_dr(setting=i, seed=seed, risk_dict=risk_dict.copy(), data=df.copy(), corr_type='cosine', irvsdr=irvsdr, k_cv=k_cv))

col_names = ['pwin', 'trdc', 'mdl', 'sm', 'cal', 'fold']+[b+a for a in ['_40_dr', '_40_ir'] for b in ['weak', 'moder', 'strong', 'vstrng']]
new_eval_data = pd.DataFrame(eval_data, columns=col_names)
f_name = f'processed_data/sepsis/eval_modeling/exp_corr_btw_fs_r_{seed}_cos_irvsdr{irvsdr}.csv'
new_eval_data.to_csv(f_name, index=False)

new_eval_data = pd.read_csv(f_name)
new_eval_data[col_names[-8:]].astype(float).mean()

100%|██████████| 1920/1920 [21:07<00:00,  1.51it/s]


weak_40_dr      0.990670
moder_40_dr     0.959085
strong_40_dr    0.892843
vstrng_40_dr    0.729684
weak_40_ir      0.996849
moder_40_ir     0.977609
strong_40_ir    0.923053
vstrng_40_ir    0.762870
dtype: float64

# EON